In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model 
import os, shutil,pathlib,exifread
import pandas as pd
import numpy as np


# Define a function that utilistes our colour image classificaiton model, input is the file name to be predicted.
def sp_predict_colour(image_name):
    
    # Load the model weights
    new_model = load_model("Species_Classification_Model.h5")
    
    # Define the image size
    image_size = (200,200)
    
    # define the file path for the image to be predicted image_name is defined by the input from the function
    test_image = os.path.abspath(image_dir/image_name)
                                 
    # Preprocess the image to the same format as the images used when training the image
    img = keras.preprocessing.image.load_img(
    test_image,target_size = image_size)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array,0)
                                 
    # Store the predicted output                             
    pred = new_model.predict(img_array) 
    list = pred.flatten()
                                 
    # Multiply the scores to change them to a percentage                                 
    probability = list*100
    comp = {spp_list[i]:probability[i] for i in range(len(spp_list))}
    dic = comp
    
    # Store all of the scores into a dataframe
    df = pd.DataFrame(probability,spp_list,columns=["Species Match (%)"])
    species = df.sort_values(by=["Species Match (%)"], ascending=False).round(2)
    
    # Return the dataframe as the output of the defined function 
    return species

# Define a function that utilises the black and white image classification model, input is the file name to be predicted.
def sp_predict_bw(image_name):
    
    # Load the model weights
    new_model = load_model("Species_Classification_Model_Gray.h5")
    
    # Define the image size + ,1 on the end to specify the inputted image will not be RGB scale
    image_size = (200,200,1)
    
    # Define the image file path (image_name) from the input of the function 
    test_image = os.path.abspath(image_dir/image_name)
    
    # Load the image and apply the same preprocessing parameters that were used in the original model 
    img = keras.preprocessing.image.load_img(
    test_image,target_size = image_size)
    
    # Converts the image to greyscale if it wasn't already
    img = tf.image.rgb_to_grayscale(img)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array,0)
    
    # Store the predicted output
    pred = new_model.predict(img_array) 
    list = pred.flatten()
    
    # Multiply by 100 to show the percentage
    probability = list*100
    comp = {spp_list[i]:probability[i] for i in range(len(spp_list))}
    dic = comp
    
    # Store the output scores into a dataframe
    df = pd.DataFrame(probability,spp_list,columns=["Species Match (%)"])
    species = df.sort_values(by=["Species Match (%)"], ascending=False).round(2)
    
    # Return the scores dataframe as the output of the function. 
    return species

# Loop through the directory and use megadetector to detect animals and crop bounding boxes to save into a new directory. 

# Define the working directory path.
cwd = pathlib.Path("test_data")
root_dir = pathlib.Path("training_data/raw_images")

# Create an autonmous list that stores the names of all species in the training data
spp_list = []
for species in os.listdir(root_dir):
    spp_list.append(species)


# Create a dataframe that contains
my_data_df = pd.DataFrame()
df_columns = {"Date":(),"Location":(),"Feed":()}

for species in os.listdir(root_dir):
    df_columns.update({species:0})
for key in df_columns:
    my_data_df[key] = df_columns[key]

    
for location in os.listdir(cwd):
    #print(location,":")
    feed_type_dir = cwd/location
    for feed_type in os.listdir(feed_type_dir):
        date_dir = feed_type_dir/feed_type/"date"
        for date in os.listdir(date_dir):
            image_dir = date_dir/date/"cropped_images"
            sp_count_day = {"Date":date,"Location":location,"Feed":feed_type}
            for species in os.listdir(root_dir):
                sp_count_day.update({species:0})
                sp_count_day.update({"Unknown":0})
                sp_count_day.update({"Total_Count":0})
                
            for image in os.listdir(image_dir):
                print("Testing ",image," on Colour Model")
                species = sp_predict_colour(image)
                species_pred = species.index[0]
                species_pred_score = species["Species Match (%)"].iloc[0]
                print(species_pred,species_pred_score)
                if species_pred_score <= 50:
                    print("Testing on B/W Model")
                    species_bw = sp_predict_bw(image)
                    species_pred_bw = species_bw.index[0]
                    species_pred_score_bw = species_bw["Species Match (%)"].iloc[0]
                    print(species_pred_bw,species_pred_score_bw)
                    if species_pred_score_bw <=70:
                        print("Unknown Animal") 
                        sp_count_day["Unknown"]+=1
                        sp_count_day["Total_Count"]+=1
                    elif species_pred_score_bw > 70:
                        for species in spp_list:
                            if species_pred_bw == species:
                                sp_count_day[species] +=1
                                sp_count_day["Total_Count"]+=1
                elif species_pred_score >50:
                    for species in spp_list:
                        if species_pred == species:
                            sp_count_day[species] +=1
                            sp_count_day["Total_Count"]+=1
            sp_count_day_df = pd.DataFrame(sp_count_day,index=[0])
            my_data_df = pd.concat([my_data_df,sp_count_day_df],ignore_index=True)
            print("")
        print("")
    print("")
display(my_data_df)
my_data_df.to_csv("count_&_species_breakdown.csv",index =False)


Testing  11240011.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 2s 2s/step
Eastern gray squirrel 38.87
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Common Wood-pigeon 80.35
Testing  11240023.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Eastern gray squirrel 63.51
Testing  11240024.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Eastern gray squirrel 38.03
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Blue tit 38.37
Unknown Animal
Testing  11240025.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Eastern gray squirrel 66.35
Testing  11240026.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Western roe deer 79.09
Testing  11240030.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1

Great tit 98.26
Testing  11250104.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 146ms/step
Blue tit 99.95
Testing  11250105.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 131ms/step
Eastern gray squirrel 89.08
Testing  11250106.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 146ms/step
Blue tit 98.62
Testing  11250107.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 144ms/step
Blue tit 97.71
Testing  11250108.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 78.39
Testing  11250112.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 99.7
Testing  11250114.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 142ms/step
Blue tit 99.16
Testing  11250116.JPG___crop00_md_

1/1 [==============================] - 0s 149ms/step
Blue tit 75.79
Testing  11250317.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 129ms/step
Blue tit 99.94
Testing  11250323.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 99.73
Testing  11250329.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Long-tailed tit 56.52
Testing  11250330.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Long-tailed tit 91.04
Testing  11250337.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blackbird 83.06
Testing  11250344.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 97.88
Testing  11250345.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 133m

1/1 [==============================] - 0s 144ms/step
Eurasian jay 99.93
Testing  11280669.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 146ms/step
Blue tit 33.81
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Common Wood-pigeon 94.69
Testing  11280672.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 145ms/step
Blue tit 46.84
Testing on B/W Model
1/1 [==============================] - 0s 139ms/step
Eurasian jay 64.65
Unknown Animal
Testing  11280674.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Nuthatch 36.51
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Common Wood-pigeon 58.72
Unknown Animal
Testing  11280675.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 97.18
Testing  11280679.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [

Great tit 87.32
Testing  12010152.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 95.22
Testing  12010153.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 99.32
Testing  12010155.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Coal tit 67.6
Testing  12010157.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 96.76
Testing  12010158.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 80.63
Testing  12010159.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 36.81
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Great tit 47.62
Unknown Animal
Testing  12010163.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [

...

1/1 [==============================] - 0s 141ms/step
Western roe deer 74.12
Testing  12200067.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 64.36
Testing  12200068.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Western roe deer 54.09
Testing  12200069.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 96.04
Testing  12200070.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Eastern gray squirrel 30.28
Testing on B/W Model
1/1 [==============================] - 0s 141ms/step
Common Wood-pigeon 63.92
Unknown Animal
Testing  12200071.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Great tit 99.98
Testing  12200072.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/st

1/1 [==============================] - 0s 156ms/step
Blue tit 48.05
Testing on B/W Model
1/1 [==============================] - 0s 156ms/step
Nuthatch 82.99
Testing  12200136.JPG___crop01_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Great tit 93.84
Testing  12200137.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Great tit 93.72
Testing  12200138.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 86.59
Testing  12200142.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Coal tit 54.49
Testing  12200142.JPG___crop01_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 99.35
Testing  12200142.JPG___crop02_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Great tit 99.97
Testing  12200143.JPG___crop00_md_

1/1 [==============================] - 0s 141ms/step
Western roe deer 83.84
Testing  12210309.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 88.3
Testing  12210311.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 87.44
Testing  12210314.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Western roe deer 68.87
Testing  12210319.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 82.56
Testing  12210319.JPG___crop01_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Blue tit 83.29
Testing  12210320.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 90.34
Testing  12210322.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 14

1/1 [==============================] - 0s 156ms/step
Great tit 90.24
Testing  12210365.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 98.69
Testing  12210367.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 98.71
Testing  12210371.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Great tit 97.76
Testing  12210372.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 156ms/step
Blue tit 43.2
Testing on B/W Model
1/1 [==============================] - 0s 156ms/step
Blue tit 41.5
Unknown Animal

Testing  12220373.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Blue tit 99.43
Testing  12220377.JPG___crop00_md_v5a.0.0.pt.jpg  on Colour Model
1/1 [==============================] - 0s 141ms/step
Coal tit 53.13
Testing  12220378.JPG

,Date,Location,Feed,Blackbird,Blue tit,Coal tit,Common Wood-pigeon,Eastern gray squirrel,Eurasian bullfinch,Eurasian jay,Eurasian wren,European robin,Great tit,Long-tailed tit,Nuthatch,Western roe deer,Unknown,Total_Count
0,2022_11_23,C2,fat_balls,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1,2022_11_24,C2,fat_balls,0,0,0,1,4,0,1,1,0,0,0,1,2,2.0,12.0
2,2022_11_25,C2,fat_balls,3,51,4,7,6,0,3,0,7,27,2,5,6,18.0,139.0
3,2022_11_26,C2,fat_balls,0,5,2,0,2,0,0,0,0,3,1,1,0,6.0,20.0
4,2022_11_27,C2,fat_balls,0,11,1,1,1,0,0,0,0,22,1,0,1,3.0,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,2022_12_26,C4,sunflower_seed,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
134,2022_12_27,C4,sunflower_seed,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,1.0
135,2022_12_28,C4,sunflower_seed,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0
136,2022_12_29,C4,sunflower_seed,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
